Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [185]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce
from collections import namedtuple
from dataclasses import dataclass 
from copy import copy, deepcopy
from math import exp

In [186]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [187]:
num_sets=100
num_points=100
x = make_set_covering_problem(num_points, num_sets, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


In [188]:
#State=namedtuple('State', 'taken not_taken')
@dataclass
class State:
    taken:set
    not_taken:set

temperature=99.3
fitness_calls=0

test_states=[]
solutions=[]
solutions_cost=[]

STATES=7

def covered(state):
    
    res=reduce(np.logical_or, [ x.toarray()[i,:] for i in state.taken], [False for _ in range(num_points)])

    res=np.reshape(res, [num_points])
    return res

def fitness1(state):
    c=covered(state)
    n= np.count_nonzero(c==True)
    return n, -len(state.taken)

# Basic randomic tweak
def tweak1(state):
    new_state=deepcopy(state)

    act=randint(0, num_sets-1)

    new_state.taken^={act}
    new_state.not_taken^={act}
    return new_state

# Tweaks more states at the same time (feels a bit cheating since it's multiple single states)
def tweak2(state):
    global test_states
    new_state=deepcopy(state)

    act=randint(0, num_sets-1)

    for i in range(STATES):
        act=randint(0, num_sets-1)
        test_states[i].taken^={act}
        test_states[i].not_taken^={act}

    new_state.taken^={act}
    new_state.not_taken^={act}
    return new_state

# Tweaks the same states with different actions (feels a bit cheating since it's multiple single states)
def tweak3(state):
    global test_states
    new_state=deepcopy(state)

    test_states=[deepcopy(state) for _ in range(STATES)]
    act=randint(0, num_sets-1)

    for i in range(STATES):
        act=randint(0, num_sets-1)
        test_states[i].taken^={act}
        test_states[i].not_taken^={act}

    new_state.taken^={act}
    new_state.not_taken^={act}
    return new_state

# Saves the quality and state in case of a set that covers everything

def register_solution(f, state):
    global solutions, solutions_cost
    if fitness(state)[0]==num_points:
        solutions_cost.append(f)
        solutions.append(state)

# Basic implementations, only compares the number of covered
# 10 tiles
def eval0(new_state, current_state):
    global fitness_calls
    fitness_calls+=1
    
    f_new=fitness(new_state)[0]
    f_cur=fitness(current_state)[0] 
    return f_new>f_cur

# Evaluates as quality the number of covered - the tiles needed to cover
# Registers solution if one is found
# 7 tiles
def eval1(new_state, current_state):
    global fitness_calls
    fitness_calls+=1

    f_new=fitness(new_state)[0] + fitness(new_state)[1]
    f_cur=fitness(current_state)[0] + fitness(current_state)[1]

    register_solution(f_new, new_state)
    return f_new>f_cur

# In early steps, if it finds a tile that covers more than the tiles already taken it substitutes the current state
# The cost in this case is avaluated just as the number of covered  
# This works with 11 tiles
def eval_best_action(new_state, current_state):
    global fitness_calls
    fitness_calls+=1
    temp_state=State(new_state.taken-current_state.taken, new_state.not_taken | current_state.not_taken)
    
    f_temp=fitness(temp_state)[0] 
    f_new=fitness(new_state)[0] 
    f_cur=fitness(current_state)[0] 

    if f_temp>f_new and f_temp>f_cur:
        return 2
    
    return f_new>f_cur

# Like above but registers early solutions and uses quality = covered - tiles taken
# 7 tiles
def eval_best_action_2(new_state, current_state):
    global fitness_calls
    fitness_calls+=1
    temp_state=State(new_state.taken-current_state.taken, new_state.not_taken | current_state.not_taken)
    
    f_temp=fitness(temp_state)[0] + fitness(temp_state)[1] 
    f_new=fitness(new_state)[0] +fitness(new_state)[1] 
    f_cur=fitness(current_state)[0] + fitness(current_state)[1] 

    register_solution(f_new, new_state)

    if f_temp>f_new and f_temp>f_cur:
        return 2
    
    return f_new>f_cur

# Evaluates more sets at the same time and returns the index of the best
# 8 tiles (works only with tweak3)
def eval_multiple(new_state, current_state):

    f_test=[fitness(test_states[i])[0]  for i in range(STATES)]

    f_test_1=[fitness(test_states[i])[1]  for i in range(STATES)]
    f_cur=fitness(current_state)[0] 
    f_cur_1=fitness(current_state)[1] 

    argmax=np.argmax(f_test)

    if(max(f_test)>f_cur and f_test_1[argmax]<f_cur_1):
        return argmax
    else:
        return -1

# Evaluates more sets at the same time and returns the index of the best
# Registers early solutions
# 7 tiles
def eval_multiple_2(new_state, current_state):
    f_test_covered=[fitness(test_states[i])[0]  for i in range(STATES)]
    f_test=[fitness(test_states[i])[0]+ fitness(test_states[i])[1] for i in range(STATES)]
    f_cur_covered=fitness(current_state)[0]
    f_cur=fitness(current_state)[0] +fitness(current_state)[1] 

    argmax=np.argmax(f_test)
    
    for i in range(STATES):
        register_solution(f_test[i], test_states[i])
    
    if(max(f_test)>f_cur):
        return argmax
    else:
        return -1

# 9 tiles
def eval_sim_annealing(new_state, current_state):
    global fitness_calls
    fitness_calls+=1
    r=random()
    f_new=fitness(new_state)[0] + fitness(new_state)[1]
    f_cur=fitness(current_state)[0] + fitness(current_state)[1]

    if(temperature<=0):
        return f_new>f_cur
    p=exp(-(f_cur-f_new)/temperature)

    return (f_new>f_cur or r<p)

def print_state(state):
    c=covered(state)
    str=""
    for i in range(num_points):
        str+="*" if c[i] else "_"
    print(str)

fitness=fitness1
tweak=tweak1
eval=eval_best_action_2
current_state=State(set(),set(range(0,num_sets)))

test_states=[deepcopy(current_state) for _ in range(STATES)]

print_state(current_state)

for i in range(num_sets):
    new_state=tweak(current_state)
    # for i in range(5):
    #     print_state(test_states[i]) 
    #print(current_state)
    #print(fitness(new_state)[0], " ",fitness(current_state)[0])
    temperature-=0.1

    val=eval(new_state, current_state)
    if(eval==eval_multiple):
        if(val>=0):
            current_state=test_states[val]

    elif(val==2):
        print("val 2")
        current_state=State(new_state.taken-current_state.taken, new_state.not_taken | current_state.not_taken)
    elif(val):
        #print_state(new_state)
        current_state=new_state
        print_state(current_state)
        

if(len(solutions)!=0):

    argmax=np.argmax(solutions_cost)
    print(argmax)
    print_state(solutions[argmax])
    print("Completed in ",i," steps with length ", len(solutions[argmax].taken))    
    print("Evaluation calls: ", fitness_calls)
else:
    print_state(current_state)
    if fitness(current_state)[0]==num_points:
        print("Completed in ",i," steps with length ", len(solutions[argmax].taken))    
        print("Evaluation calls: ", fitness_calls)      
    
    


____________________________________________________________________________________________________
*_*_*_____**_____*_**___*___*__*_**_*_____*__*_*____*_________**_*__*______*_**___*__***____*___*___
*******___**_____*_**_***___**_*_**_*___***__*_**__**_*_*_*___**_*__*__*___*_**__***_***___***_**___
*******__***_____*_**_***_*_**_*_**_*___****_*_**__****_*_*__***_*__*_**_*_*****_*******___******___
*******__****____****_********_****_*__**********_*****_*_*__***_*******_***************__*******_*_
*******__*********************_******__**********_*****_*_*__***_************************_**********
******************************_******__****************_*_*__****************************_**********
*******************************************************_*_*__****************************_**********
************************************************************_***************************************
************************************************************_******************************